In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd 'drive/My Drive/AI projects/VillagePlantClassification'

/content/drive/My Drive/AI projects/VillagePlantClassification


In [3]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head()

,image_file,label,width,height
0,0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Sp...,Pepper__bell___Bacterial_spot,256,256
1,006adb74-934f-448f-a14f-62181742127b___JR_B.Sp...,Pepper__bell___Bacterial_spot,256,256
2,00f2e69a-1e56-412d-8a79-fdce794a17e4___JR_B.Sp...,Pepper__bell___Bacterial_spot,256,256
3,01613cd0-d3cd-4e96-945c-a312002037bf___JR_B.Sp...,Pepper__bell___Bacterial_spot,256,256
4,0169b9ac-07b9-4be1-8b85-da94481f05a4___NREC_B....,Pepper__bell___Bacterial_spot,256,256


In [4]:
# Data analysing
len(data)

20638

In [5]:
# Label counts
data.label.value_counts()

Tomato__Tomato_YellowLeaf__Curl_Virus          3208
Tomato_Bacterial_spot                          2127
Tomato_Late_blight                             1909
Tomato_Septoria_leaf_spot                      1771
Tomato_Spider_mites_Two_spotted_spider_mite    1676
Tomato_healthy                                 1591
Pepper__bell___healthy                         1478
Tomato__Target_Spot                            1404
Tomato_Early_blight                            1000
Potato___Late_blight                           1000
Potato___Early_blight                          1000
Pepper__bell___Bacterial_spot                   997
Tomato_Leaf_Mold                                952
Tomato__Tomato_mosaic_virus                     373
Potato___healthy                                152
Name: label, dtype: int64

In [6]:
LABELS = data['label'].unique()
LABELS

array(['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
       'Tomato_Late_blight', 'Tomato_Leaf_Mold',
       'Tomato_Septoria_leaf_spot',
       'Tomato_Spider_mites_Two_spotted_spider_mite',
       'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
       'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'], dtype=object)

In [7]:
# Split train test
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size= 0.2, random_state = 2020)

In [8]:
# ImageGenerator object
import tensorflow as tf
import keras
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [9]:
TRAIN_DIR = 'train'
IMAGE_SIZE = 256
BATCH_SIZE = 32

train_gen = train_datagen.flow_from_dataframe(dataframe= train_data,
                                         directory= TRAIN_DIR,
                                         x_col= 'image_file',
                                         y_col= 'label',
                                        target_size= (IMAGE_SIZE, IMAGE_SIZE),
                                         class_mode= 'categorical',
                                         batch_size= BATCH_SIZE,
                                         shuffle= True)

val_gen = val_datagen.flow_from_dataframe(dataframe= val_data,
                                         directory= TRAIN_DIR,
                                         x_col= 'image_file',
                                         y_col= 'label',
                                        target_size= (IMAGE_SIZE, IMAGE_SIZE),
                                         class_mode= 'categorical',
                                         batch_size= BATCH_SIZE,
                                         shuffle= False)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image_file". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 16508 validated image filenames belonging to 15 classes.
Found 4128 validated image filenames belonging to 15 classes.


In [10]:
# Build model
from keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dropout

model = keras.Sequential([
                          Conv2D(filters= 32, kernel_size= (3,3), padding= 'same', input_shape= (256, 256, 3), activation= 'relu'),
                          BatchNormalization(),
                          MaxPool2D((3,3)),
                          Dropout(0.25),
                          Conv2D(filters= 64, kernel_size= (3,3), padding= 'same', activation= 'relu'),
                          BatchNormalization(),
                          Conv2D(filters= 64, kernel_size= (3,3), padding= 'same', activation= 'relu'),
                          BatchNormalization(),
                          MaxPool2D((3,3)),
                          Dropout(0.25),
                          Conv2D(filters= 128, kernel_size= (3,3), padding= 'same', activation= 'relu'),
                          BatchNormalization(),
                          Conv2D(filters= 128, kernel_size= (3,3), padding= 'same', activation= 'relu'),
                          BatchNormalization(),
                          MaxPool2D((3,3)),
                          Dropout(0.25),
                          Flatten(),                                                                          
                          Dense(1024, activation= 'relu'),
                          Dropout(0.25),
                          Dense(len(LABELS), activation= 'softmax')                                                   
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 85, 85, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        3

In [11]:
# Training phase
LR = 1e-3
optimizer = keras.optimizers.Adam(learning_rate= LR)
model.compile(optimizer= optimizer, loss= 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
NUM_EPOCH = 10
model.fit_generator(generator= train_gen,
                    steps_per_epoch= train_gen.n //BATCH_SIZE,
                    epochs= NUM_EPOCH,
                    verbose= 1,
                    validation_data= val_gen,
                    validation_steps= val_gen.n // BATCH_SIZE,
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 13/515 [..............................] - ETA: 1:09:39 - loss: 1.5980 - accuracy: 0.1010